In [12]:
# AI model to classify space rocks

Matplotlib. Use this library to plot your data. Add the following code in new cell in your Jupyter Notebook file, and then run the code.
NumPy library to process large numerical matrixes (images), and run the new cell.

In [13]:
import matplotlib.pyplot as plt
import numpy as np

PyTorch library to train and process deep learning and AI models. Torchvision, which is part of PyTorch. Use this library to process images and do manipulations like cropping and resizing.

In [14]:
import torch
from torch import nn, optim
from torch.autograd import Variable
import torch.nn.functional as F

import torchvision
from torchvision import datasets, transforms, models

Python Imaging Library (PIL) to visualize the images

In [15]:
from PIL import Image

Two libraries that ensure the plots are shown inline and with high resolution. 

In [16]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [17]:
# Tell the machine what folder contains the image data
data_dir = './Data'

# Read the data, crop and resize the images, split data into two groups: test and train
def load_split_train_test(data_dir, valid_size = .2):

    # Transform the images to train the model
    train_transforms = transforms.Compose([
                                       transforms.RandomResizedCrop(224),
                                       transforms.Resize(224),
                                       transforms.ToTensor(),
                                       ])

    # Transform the images to test the model
    test_transforms = transforms.Compose([transforms.RandomResizedCrop(224),
                                          transforms.Resize(224),
                                          transforms.ToTensor(),
                                      ])

    # Create two variables for the folders with the training and testing images
    train_data = datasets.ImageFolder(data_dir, transform=train_transforms)
    test_data = datasets.ImageFolder(data_dir, transform=test_transforms)

    # Get the number of images in the training folder
    num_train = len(train_data)

    # Create a list of numbers from 0 to the number of training images - 1
    # Example: For 10 images, the variable is the list [0,1,2,3,4,5,6,7,8,9]
    indices = list(range(num_train))

    # If valid_size is .2, find the index of the image that represents 20% of the data
    # If there are 10 images, a split would result in 2
    # split = int(np.floor(.2 * 10)) -> int(np.floor(2)) -> int(2) -> 2
    split = int(np.floor(valid_size * num_train))

    # Randomly shuffle the indices
    # For 10 images, an example would be that indices is now the list [2,5,4,6,7,1,3,0,9,8]
    np.random.shuffle(indices)

    from torch.utils.data.sampler import SubsetRandomSampler

    # With the indices randomly shuffled, 
    # grab the first 20% of the shuffled indices, and store them in the training index list
    # grab the remainder of the shuffled indices, and store them in the testing index list
    # Given our example so far, this would result is:
    # train_idx is the list [2,5] 
    # test_idx is the list [4,6,7,1,3,0,9,8]
    train_idx, test_idx = indices[split:], indices[:split]

    # Create samplers to randomly grab items from the training and testing indices lists
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(test_idx)

    # Create loaders to load 24 images from the train and test data folders
    # Images are chosen based on the shuffled index lists and by using the samplers
    trainloader = torch.utils.data.DataLoader(train_data, sampler=train_sampler, batch_size=24)
    testloader = torch.utils.data.DataLoader(test_data, sampler=test_sampler, batch_size=24)

    # Return the loaders so you can grab images randomly from the training and testing data folders
    return trainloader, testloader

# Using the function that shuffles images,
# create a trainloader to load 20% of the images
# create a testloader to load 80% of the images
trainloader, testloader = load_split_train_test(data_dir, .2)

# Print the type of rocks that are included in the trainloader
print(trainloader.dataset.classes)

['Basalt', 'Highland']


# Detect device type

determine the most efficient way to create the deep learning network. First, find the type of device you're using: CPU or GPU. The PyTorch APIs offer support to form a neural network according to the device type.

In [18]:
# OLD: Setup the model with pretrained weights and send it to the target device (this was prior to torchvision v0.13)
# model = torchvision.models.efficientnet_b0(pretrained=True).to(device) # OLD method (with pretrained=True)

# Determine if you're using a CPU or a GPU device to build the deep learning network
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = models.resnet50(pretrained=True)

# NEW: Setup the model with pretrained weights and send it to the target device (torchvision v0.13+)
weights = torchvision.models.ResNet50_Weights.DEFAULT # .DEFAULT = best available weights 
model = torchvision.models.resnet50(weights=weights).to(device)

#model # uncomment to output (it's very long)

# Build neurons and wire the network

Build the neurons and wire the network.

In [19]:
# Build all the neurons
for param in model.parameters():
     param.requires_grad = False

# Wire the neurons together to create the neural network
model.fc = nn.Sequential(nn.Linear(2048, 512),
                               nn.ReLU(),
                               nn.Dropout(0.2),
                               nn.Linear(512, 2),
                               nn.LogSoftmax(dim=1))

criterion = nn.NLLLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.003)

# Add the neural network to the device
model.to(device)

print('done')

done


# Train a neural network to accurately classify space rocks in photos
## Iterate on the data and increase the accuracy
In this section of code, look for the epochs variable. This variable tells the program how many times to search for associations in the features. In our example, we'll set the initial number of iterations to 10.

To train our model, we load the image input from the trainloader variable that we built in the Analyze images of rocks with AI module. The data is stored to the already selected device. We call the optimizer.zero_grad() function to zero out gradients and avoid the accumulation of gradients across training iterations.

The image input is passed through the model by using the model.forward(inputs) function, which returns the log probabilities of each label. The criterion(logps, labels) function runs the log probabilities through the criterion to get the output graph. The loss.backward() function uses the loss graph to compute the gradients. The optimizer.step() function then updates the parameters based on the current gradient.

During the training and testing, we track the loss values for each iteration and the full batch. Every five epochs, we evaluate the model. We use the model.eval() function with the torch.no_grad() function to turn off parts of the model that behave differently during training versus evaluation. We use this pair of functions to refine the accuracy of the prediction without updating the gradients.

The torch.exp(logps) function is used to get a new tensor with the true probabilities. The largest probability and class of the new tensor along a given dimension is returned from the ps.topk(1, dim=1) function. The tensor is reshaped to match the same shape as the top class.

Finally, we compute the overall accuracy.

# Train the neural network

In [20]:
# Set the initial number of iterations to search for associations
epochs = 100
print_every = 20

# Initialize the loss variables
running_loss = 0
train_losses, test_losses = [], []

# Track the current training step, start at 0
steps = 0

# Search for associations in the features
for epoch in range(epochs):

   # Count each epoch
   epoch += 1

   # Load in all of the image inputs and labels from the TRAIN loader 
   for inputs, labels in trainloader:

      # Count each training step
      steps += 1
      print('Training step ', steps)

      # Load the inputs and labels to the already selected device
      inputs, labels = inputs.to(device), labels.to(device)

      # Zero out gradients to avoid accumulations of gradiants across training iterations
      optimizer.zero_grad()

      # Pass the images through the model, return the log probabilities of each label
      logps = model.forward(inputs)

      # Run the log probabilities through the criterion to get the output graph
      loss = criterion(logps, labels)

      # Use the loss graph to compute gradients
      loss.backward()

      # Update the parameters based on the current gradient
      optimizer.step()

      # Add the actual loss number to the running loss total
      running_loss += loss.item()

      # Every 5 steps, evaluate the model
      if steps % print_every == 0:

         # Initialize loss and accuracy
         test_loss = 0
         accuracy = 0

         # Start the model evaluation
         model.eval()

         # Refine the accuracy of the prediction without updating the gradients
         with torch.no_grad():

            # Load in all of the image inputs and labels from the TEST loader 
            for inputs, labels in testloader:

               # Load the inputs and labels to the already selected device
               inputs, labels = inputs.to(device), labels.to(device)

               # Pass the images through the model, return the log probabilities of each label
               logps = model.forward(inputs)

               # Run the log probabilities through the criterion to get the output graph
               batch_loss = criterion(logps, labels)

               # Add the actual loss number to the running loss total for the test batch
               test_loss += batch_loss.item()

               # Return a new tensor with the true probabilities
               ps = torch.exp(logps)

               # Return the largest probability and class of the new tensor along a given dimension
               top_p, top_class = ps.topk(1, dim=1)

               # Reshape the tensor to match the same shape as the top class
               equals = top_class == labels.view(*top_class.shape)

               # Compute the accuracy and add it to the running accuracy count for the test batch
               accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

         # Append the training and testing losses
         train_losses.append(running_loss/len(trainloader))
         test_losses.append(test_loss/len(testloader))  

         # Display the accuracy of the prediction with 3 digits in the fractional part of the decimal
         print(f"\n     Epoch {epoch}/{epochs}: "
               f"Train loss: {running_loss/print_every:.3f}.. "
               f"Test loss: {test_loss/len(testloader):.3f}.. "
               f"Test accuracy: {accuracy/len(testloader):.3f}\n")

         # Train the model
         running_loss = 0
         model.train()

         # After 10 training steps, start the next epoch
         # Break here in case the trainloader has remaining data
         break

Training step  1
Training step  2
Training step  3
Training step  4
Training step  5
Training step  6
Training step  7
Training step  8
Training step  9
Training step  10
Training step  11
Training step  12
Training step  13
Training step  14
Training step  15
Training step  16
Training step  17
Training step  18
Training step  19
Training step  20

     Epoch 4/100: Train loss: 0.234.. Test loss: 0.287.. Test accuracy: 0.887

Training step  21
Training step  22
Training step  23
Training step  24
Training step  25
Training step  26
Training step  27
Training step  28
Training step  29
Training step  30
Training step  31
Training step  32
Training step  33
Training step  34
Training step  35
Training step  36
Training step  37
Training step  38
Training step  39
Training step  40

     Epoch 8/100: Train loss: 0.100.. Test loss: 0.076.. Test accuracy: 0.979

Training step  41
Training step  42
Training step  43
Training step  44
Training step  45
Training step  46
Training step  47
Tra

# Analyze the training output
After five epochs are complete, the system reaches our epoch limit.

The output shows the prediction accuracy for each epoch iteration with training and testing losses, and the test accuracy.

Here are the results from our test with five epochs. Your specific results will differ because the computer chooses a set of random images for each test run. The results reveal the training loss, testing loss, and accuracy, all depend on the chosen image.

| Epoch	| Training loss	| Test loss	| Test accuracy |
| ----- | ------------- | --------- | ------------- |
| 1	    | 0.550	        | 0.282	    | 0.902         |
| 2	    | 0.451	        | 0.311	    | 0.842         |
| 3	    | 0.342	        | 0.233	    | 0.902         |
| 4	    | 0.216	        | 0.189	    | 0.906         |
| 5	    | 0.234	        | 0.175	    | 0.935         |

In [21]:
# The following code calculates and displays the accuracy of our AI model to classify the rock type.
print(accuracy/len(testloader))

0.9375


In [22]:
# Saving the model...
torch.save(model, 'aerialmodel.pth')

# Use the model to make predictions

Steps to make predictions with the neural network in your AI model.

In [23]:
# First, load the neural network
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model=torch.load('aerialmodel.pth')

In [24]:
# create a function to predict the type of rock in a new image by comparing it against our model's matrix pattern
def predict_image(image):
    image_tensor = test_transforms(image).float()
    image_tensor = image_tensor.unsqueeze_(0)
    input = Variable(image_tensor)
    input = input.to(device)
    output = model(input)
    index = output.data.cpu().numpy().argmax()
    return index